kgmyh@naver.com 으로 `이름_주유소가격 데이터 분석.html` 로 제출
- html 파일로 변환
    - File > Download as > HTML (.html) 선택

# 데이터 불러오기

- 2022년 주유소들의 일자별 가격 데이터
- https://www.opinet.co.kr/user/opdown/opDownload.do

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'malgun gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 상반기, 하반기 주유소 가격 데이터 조회
d1 = pd.read_csv('data/2022년_서울_상반기_일별_가격.csv', encoding='cp949')
d2 = pd.read_csv('data/2022년_서울_하반기_일별_가격.csv', encoding='cp949')
d1.shape, d2.shape

In [ ]:
# 상반기, 하반기 DataFrame 합치기
df = pd.concat([d1, d2], ignore_index=True)
df.shape

In [ ]:
# DataFrame 기본 정보 조회
df.info()

In [ ]:
df.head()

In [ ]:
df.tail(10)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()  #  정수/실수 타입 컬럼

In [ ]:
df.describe(include='object') # 문자열(object) 

In [ ]:
# 상표 컬럼의 고유값 조회
df['상표'].unique()
df['상표'].value_counts()
df['상표'].value_counts(normalize=True)

In [ ]:
# 지역 컬럼의 고유값 조회
df['지역'].unique()
df['지역'].value_counts()
df['지역'].value_counts(normalize=True)

In [ ]:
df['셀프여부'].value_counts()

> ### str accessor
> - `Series.str.String메소드()`
> - 문자열 타입 Series에 String 메소드들을 사용하여 일괄처리 할 수 있도록 한다.
> - str accessor 메소드 : https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-str

> ### datetime으로 변환
> - `pd.to_datetime(변환할 Series, format="날짜형식")`
> - x 또는 y 축이 날짜 일 경우 datetime 타입으로 처리하는 것이 좋다.
> - format: 년월일의 형식을 strftime 형식문자열로 지정한다. 
>      - [strftime 형식](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)  
> - datetime을 변환하면 날짜만의 기능을 다룰 수 있다.
>     - dt accessor를 이용해 datetime의 속성값들을(년, 월, 일 등) 추출할 수 있다.
>     - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html


In [ ]:
# 방법1: 지역 컬럼의 값을 이용하여 "구" 컬럼 생성 (지역에서 구만 추출해서 파생변수컬럼 생성)
df['구'] = df['지역'].apply(lambda x : x.split()[1])
df.head()

In [ ]:
 # 방법2: str accessor 이용
df['지역'].str.split(expand=True)[1] # expand=True split후 두개 컬럼별로 나눠준다.

In [ ]:
# 기간 컬럼을 datetime 타입으로 변경
df['기간'] = pd.to_datetime(df['기간'], format="%Y%m%d")  #Y:년, m:월(2자리), d:일,  H: 시간, M:분, S: 초

In [ ]:
df.info()

In [ ]:
# 기간 컬럼을 이용해 "월", "일", "요일" 컬럼 생성 (dt accessor 이용)
df['월'] = df['기간'].dt.month
df['일'] = df['기간'].dt.day
df['요일'] = df['기간'].dt.weekday

In [ ]:
df.head()

In [ ]:
# 요일 - 0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일 로 변경
df['요일'] = df['요일'].apply(lambda x : "월화수목금토일"[x])# +"요일")

In [ ]:
df.head()

In [ ]:
# 전처리를 완료 -> 파일로 저장
df.to_csv('data/2022년_주유소_가격_데이터_preprocessing.csv', index=False)

In [ ]:
df = pd.read_csv('data/2022년_주유소_가격_데이터_preprocessing.csv')
df.head()

In [ ]:
# 휘발유 가격이 가장 비싼 5개 행 조회
# df.sort_values('휘발유', ascending=False).head()
df.sort_values(['휘발유', '기간'], ascending=[False, True]).head()

In [ ]:
##### 방법2 - dfataframe.nlargest(개수, 컬럼) 컬럼기준으로 가장 큰순서대로 개수 행 만큼 반환.  dataframe.nsmallest(개수, 컬럼) 작은 순서대로 반환
df.nlargest(5, '휘발유')

In [ ]:
### 휘발유 가격 평균을 기준으로 비싼 주유소 5개 조회
df.groupby('상호')['휘발유'].mean().nlargest(5)

In [ ]:
# 휘발유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)
df.query('휘발유 != 0').sort_values(['휘발유', '기간']).head(5)

In [ ]:
### 방법2:
df.query('휘발유 != 0').nsmallest(5, '휘발유')

In [ ]:
# 고급휘발유 가격이 가장 비싼 5개 행 조회
df.sort_values(['고급휘발유', '기간'], ascending=[False, True]).head()

In [ ]:
# 고급휘발유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)
df.query('고급휘발유 != 0').sort_values(['고급휘발유', '기간']).head()

In [ ]:
# 경유 가격이 가장 비싼 5개 행 조회
df.sort_values(['경유', '기간'], ascending=[False, True]).head()

In [ ]:
# 경유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)
df.query('경유 != 0').sort_values(['경유', '기간']).head()

In [ ]:
# 상표별 휘발유 평균가격 조회
상표별_휘발유_평균가격 = df.query('휘발유!=0').groupby('상표')['휘발유'].mean().sort_values() # 0원인 주유소는 빼고 계산
상표별_휘발유_평균가격 

In [ ]:
# 상표별 휘발유 평균가격 시각화
휘발유_전체_평균 = np.round(df['휘발유'].mean(), 2)

상표별_휘발유_평균가격.plot(kind='bar', rot=45)

plt.axhline(휘발유_전체_평균, color='red', linestyle=':', label=f'전체평균:\u20A9{휘발유_전체_평균:.2f}', alpha=0.5)  # 평균가격에 수평선 긋기 (axvline(): 수직선)

for i in range(상표별_휘발유_평균가격.size):
    txt = str(np.round(상표별_휘발유_평균가격.iloc[i]))
    plt.text(i-0.3, 상표별_휘발유_평균가격.iloc[i]+10, txt)

plt.title('상표별 휘발유 평균가격')
plt.ylim(1000, 2050)
plt.legend(bbox_to_anchor=(1,1), loc='upper left')
plt.show()

In [ ]:
# 셀프주유소와 일반주유소 개수 확인
df['셀프여부'].value_counts()
# df['셀프여부'].value_counts(normalize=True)  # 비율로 조회

In [ ]:
# 셀프 여부에 따른 휘발유 평균 가격 비교 
df.query("휘발유 != 0").groupby('셀프여부')['휘발유'].mean()

In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격 평균 비교
상표_셀프여부별_휘발유_평균 = df.pivot_table(index='상표', columns='셀프여부', values='휘발유', aggfunc='mean', margins=True)
상표_셀프여부별_휘발유_평균

In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격 평균 비교 시각화

## all(margin)은 그리지 않기 위해 조회에서 뺀다. (iloc[:-1, :-1])
상표_셀프여부별_휘발유_평균.iloc[:-1, :-1].plot(
    kind='bar', 
    title='상표, 셀프여부별 휘발유 평균가격 비교', 
    ylim=(1200, 2100), 
    figsize=(10,7), 
    # stacked=True,
    rot=0
)

plt.show()

In [ ]:
#  휘발유 가격 boxplot
df.query('휘발유!=0')['휘발유'].plot(kind='box')
plt.show()

In [ ]:
# 상표별 휘발유 가격의 분포를 boxplot으로 시각화
### seaborn이용: group 별로 나눠서 그리는 것은 seaborn이 편리
plt.figure(figsize=(10,5))

sns.boxplot(data=df.query('휘발유!=0'), y='휘발유', x='상표')  # y: 연속형 -> 4분위, 정상범위 등을 계산,  x: 범주형 - 나누는 대상
# sns.violinplot(data=df.query('휘발유!=0'), y='휘발유', x='상표')  # violinplot()은 boxplot에 histgram을 추가하여 시각화한다.

plt.title('상표별 휘발유 가격 분포')
plt.show()

In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격에 대한 분포 확인 - boxplot으로 시각화
plt.figure(figsize=(13,7))
sns.boxplot(data=df.query('휘발유 != 0'), y='휘발유', x='상표', hue='셀프여부')  # hue: 범주형 -> 범주값에 따라 다른 색으로 구분해서 그린다. (그룹별로 나눠서 시각화 할 때 사용)
# sns.violinplot(data=df.query('휘발유 != 0'), y='휘발유', x='상표', hue='셀프여부')
plt.show()

In [ ]:
# 구별 휘발유 가격의 평균
result = df.query('휘발유 != 0').groupby('구')['휘발유'].mean().sort_values(ascending=False)
result

In [ ]:
# 평균 가격이 가장 높은 구의 상표별 휘발유 평균가격
df.loc[df['구']==result.index[0]].groupby('상표')['휘발유'].mean()

In [ ]:
# 월 평균 휘발유 가격
월별_평균_휘발유_가격 = df.query('휘발유!=0').groupby('월')['휘발유'].mean().sort_index()
월별_평균_휘발유_가격

In [ ]:
# 월 평균 휘발유 가격  변화추이를 시각화
월별_평균_휘발유_가격.plot(figsize=(15, 4), marker='.')
plt.xticks(range(1, 13), labels=[str(i)+'월' for i in range(1,13)]) 
plt.grid(True)
plt.show()

In [ ]:
pd.options.display.max_columns = 30

In [ ]:
# 각 구의 월별 평균 휘발유 가격 
구_월별_휘발류_평균가격 = round(df.pivot_table(index='월', columns='구', values='휘발유', aggfunc='mean'), 2)
구_월별_휘발류_평균가격

In [ ]:
# 각 구의 월별 평균가격의 변화추이 시각화
구_월별_휘발류_평균가격.plot(
    figsize=(15, 5), 
    title='각 구의 월별 평균가격의 변화추이', 
    ylabel='평균가격', 
)
plt.legend(bbox_to_anchor=(1,1), loc='upper left', ncol=2, title='구')
plt.show()

In [ ]:
# 가장 휘발유가격이 비싼 주유소 상위 5 - (같은 주유소가 여러개 나오므로 가장 비싼 가격을 기준으로 집계)

# 상호, 주소로 그룹을 나눔: 주소: 상호가 같은 주유소가 있을 경우를 대비
비싼주유소 = df.groupby(['상호', '주소'])['휘발유'].max().sort_values(ascending=False).head(5)
비싼주유소

In [ ]:
# 가장 휘발유가격이 저렴한 주유소 상위 5 - (같은 주유소가 여러개 나오므로 가장 비싼 가격을 기준으로 집계)

### 주유소별 휘발유 가장 비쌌을때 가격을 기준으로 조회
저렴한주유소 = df.query('휘발유!=0').groupby(['상호', '주소'])['휘발유'].max().sort_values().head(5)
저렴한주유소

In [ ]:
# 휘발유 비싼 주유소의 상위 5개의 상호, 지역, 주소 조회(중복제거-drop_duplicates)

# multiindex.get_level_values(level): 멀티 인덱스에서 level의 index(행/컬럼 이름)을 조회
df.loc[df['상호'].isin(비싼주유소.index.get_level_values(0)) &  
       df['주소'].isin(비싼주유소.index.get_level_values(1)), ['상호', '지역', '주소']].drop_duplicates()

In [ ]:
# 휘발유 저렴한 주유소의 상위 5개의 상호, 지역, 주소 조회(중복제거)
##  query() 이용.
df.query("상호 in @저렴한주유소.index.get_level_values(0) and 주소 in @저렴한주유소.index.get_level_values(1)")[['상호', '지역', '주소']].drop_duplicates()

In [ ]:
# 고급 휘발유를 가장 많이 파는 구 조회
df.query('고급휘발유!=0').groupby('구')['고급휘발유'].count().sort_values(ascending=False)  #동일한 주유소가 같이 count가 됨.

In [ ]:
## 중복된 컬럼 빼고 조회
df.query('고급휘발유!=0')[['구','상호','주소']].drop_duplicates().groupby('구').count()['상호'].sort_values(ascending=False)

In [ ]:
###############################################################################
# 주유소 가격 데이터를 이용해서 알고 싶은 정보들을 추가 분석해 보세요.
###############################################################################